In [3]:
import openai

# openai.api_base = "https://api.openai.com/v1" # 换成代理，一定要加v1
openai.api_base = "https://openkey.cloud/v1" # 换成代理，一定要加v1
# openai.api_key = "API_KEY"
openai.api_key = "sk-J6LW3tnRjWJhZtQiB69bE9C3E7554594A6F45f21884a0a7a"


In [16]:
PROMPT_TEMPLATE='''Given a sentence and some entities connnected by periods, you are required to ask some relevant questions about the specified entities involved in the sentence, so that the questions can help to verify the factuality of the sentence.
Questions may involve basic attributes such as colors, actions mentioned in the sentence. Do not ask questions involving object counts or the existence of object.
When asking questions about attributes, try to ask simple questions that only involve one entity. 
When asking questions about positional relationships, must provide specific positional relationships between objects or the positional relationships of objects in the image.
Ask questions that can be easily decided visually. Do not ask questions that require complex reasoning.
Do not ask semantically similar questions. Do not ask questions only about scenes or places.
Do not ask questions about uncertain or conjecture parts of the sentence, for example, the parts described with "maybe" or "likely", etc.
It is no need to cover all the specified entities. If there is no question to ask, simply output a 'None'.
When asking questions, do not assume the claims in the description as true in advance. Only ask questions relevant to the information in the sentence.
Only ask questions about common, specific and concrete entities. The entities involved in the questions are limited to the range within the given entities.
Output only one question in each line. For each line, first output the question, then a single '&', and finally entities involved in the question, still connected by periods if multiple entities are involved. The order of entities needs to be consistent with the order in which they appear in the questions.

Examples:
Sentence:
There are one black dog and two white cats in the image.

Entities:
dog.cat

Questions:
What color is the cat?&cat
What color is the dog?&dog

Sentence:
The man is wearing a baseball cap and appears to be smoking.

Entities:
man

Questions:
What is the man wearing?&man
What is the man doing?&man

Sentence:
The image depicts a busy kitchen, with a man in a white apron. The man is standing in the middle of the kitchen.

Entities:
kitchen.man

Questions:
What does the man wear?&man
Is the man standing in the middle of the kitchen?&man.kitchen

Sentence:
The trash can is under the cup in the image.

Entities:
trash can.cup

Questions:
Is the trash can under the cup in the image?&trash can.cup

Sentence:
The white mouse is on the left of the keyboard.

Entities:
mouse.keyboard

Questions:
What color is the mouse?&mouse
Is the white mouse on the left of the keyboard?&mouse.keyboard

Sentence:
{sent}

Entities:
{entity}

Questions:'''

In [17]:
# import openai

# # openai.api_base = "https://api.openai.com/v1" # 换成代理，一定要加v1
# openai.api_base = "https://openkey.cloud/v1" # 换成代理，一定要加v1
# # openai.api_key = "API_KEY"
# openai.api_key = "sk-J6LW3tnRjWJhZtQiB69bE9C3E7554594A6F45f21884a0a7a"

In [18]:
def remove_duplicates(res):
    qs_set = set()
    output = []
    for s in res:
        qs, ent = s
        if qs in qs_set:
            continue
        else:
            output.append(s)
            qs_set.add(qs)
    return output

In [19]:
sent = "yes, the blue umbrella is under the black umbrella in the image."
entity = "umbrella"


content = PROMPT_TEMPLATE.format(sent=sent, entity=entity)

response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[{
        'role': 'system',
        'content': 'You are a language assistant that helps to ask questions about a sentence.'
    }, {
        'role': 'user',
        'content': content,
    }],
    temperature=0.2,
    max_tokens=1024,
)
res = response['choices'][0]['message']['content'].splitlines()
res

['What color is the umbrella?&umbrella',
 'Is the blue umbrella under the black umbrella in the image?&umbrella.umbrella']

In [7]:
res = response['choices'][0]['message']['content'].splitlines()
res = [s.split('&') for s in res if s.lower() != 'none']
entity_list = entity.split('.')

res = [s for s in res if len(s)==2]
res = remove_duplicates(res)
res = [s for s in res if set(s[1].split('.')).issubset(set(entity_list)) ]


In [8]:
res

[['Is the baby on the right of the dog in the image?', 'baby.dog']]

In [4]:
PROMPT_TEMPLATE='''Given a sentence, extract the entities it contains.
Extract the common entities in the singular form.
The entities should be general categories.
Do not extract entities that are non-specific or abstract.
Merge similar entities as much as possible.
Output all identified entities in a single line, using "||" as a separator between each entity.
If there is no entity extracted, the output should be simply "None".

Examples:
Sentence:
The image depicts a man laying on the ground next to a motorcycle, which appears to have been involved in a crash.

Entities:
man||motorcycle

Sentence:
There are a few people around, including one person standing close to the motorcyclist and another person further away.

Entities:
person||motorcyclist

Sentence:
No, there is no car in the image.

Entities:
car

Sentence:
The image depicts a group of animals, with a black dog, a white kitten, and a gray cat, sitting on a bed.

Entities:
dog||cat||bed

Sentence:
{sentence}

Entities:'''

In [5]:
sentence="yes, the blue umbrella is under the black table in the image."
content = PROMPT_TEMPLATE.format(sentence=sentence)

response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[{
        'role': 'system',
        'content': 'You are a language assistant that assists in extracting entities from the sentences provided.'
    }, {
        'role': 'user',
        'content': content,
    }],
    temperature=0.2,
    max_tokens=1024,
)
res = response['choices'][0]['message']['content'].splitlines()
res

['umbrella||table']